In [ ]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_decoder = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer.pad_token = "<PAD>"


In [ ]:
caption_length = 20 


caption = "Who is Modi "

tokens = tokenizer(
            caption,
            max_length=caption_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


In [ ]:
tokens

In [ ]:
import torch 

gpt_decoder.eval() 

with torch.no_grad():
        output_ids = gpt_decoder.generate(
            input_ids=tokens["input_ids"],
            max_length=30,
            num_beams=3,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode
tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt 
from transformers.generation.logits_process import LogitsProcessorList
from transformers import LogitsProcessorList, MinLengthLogitsProcessor, RepetitionPenaltyLogitsProcessor

from transformers.generation.logits_process import (
    TemperatureLogitsWarper,
    TopKLogitsWarper,
    TopPLogitsWarper,
)

# Chain multiple warpers
warpers = LogitsProcessorList([
    TemperatureLogitsWarper(0.8),
    TopKLogitsWarper(50),
    TopPLogitsWarper(0.95),
])

processors = LogitsProcessorList([
    RepetitionPenaltyLogitsProcessor(penalty=1.2)
])


def greedy_generate(model, tokenizer, input_ids, max_new_tokens=30, device="cpu"):
    model.eval()
    generated = input_ids.to(device)

    for _ in range(max_new_tokens):
        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids=generated)
            logits = outputs.logits  # (B, T, V)
        
        # Get last token logits
        next_token_logits = logits[:, -1, :]  # (B, V)

        next_token_logits = processors(generated, logits[:, -1, :])

        probs = F.softmax(next_token_logits, dim=-1)
            # Sample on CPU to avoid MPS issues
        next_token_id = torch.multinomial(probs.cpu(), num_samples=1).to(device)

        # Append to sequence
        generated = torch.cat([generated, next_token_id], dim=1)

        # Stop if EOS generated
        if tokenizer.eos_token_id is not None and (next_token_id == tokenizer.eos_token_id).all():
            break

    return generated

# Example usage
tokens = tokenizer("Who is Modi ", return_tensors="pt")
output_ids = greedy_generate(gpt_decoder, tokenizer, tokens["input_ids"], max_new_tokens=30)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer 
import torch 
import os 


model_name = "Falcon3-1B-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gpt_decoder = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

In [ ]:
inputs = tokenizer(
            "Who is Modi",
            max_length=5,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


In [ ]:
inputs

In [ ]:
tokenizer.decode(inputs["input_ids"].view(-1), skip_special_tokens=False)

In [ ]:
gpt_decoder.get_input_embeddings()(inputs["input_ids"]).shape

In [ ]:
import torch 

gpt_decoder.eval() 
with torch.no_grad():
        output_ids = gpt_decoder.generate(
            input_ids=inputs["input_ids"],
            max_new_tokens = 40,
        )
tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [1]:

from transformers import GPTNeoForCausalLM, AutoTokenizer

model_name_or_path = "GPT-NEO-350M"  

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = GPTNeoForCausalLM.from_pretrained(model_name_or_path)

inputs = tokenizer("Hello, how are you?", return_tensors="pt")


/Users/preetamverma/Desktop/multimodel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt 
from transformers.generation.logits_process import LogitsProcessorList
from transformers import LogitsProcessorList, MinLengthLogitsProcessor, RepetitionPenaltyLogitsProcessor

from transformers.generation.logits_process import (
    TemperatureLogitsWarper,
    TopKLogitsWarper,
    TopPLogitsWarper,
)

# Chain multiple warpers
warpers = LogitsProcessorList([
    TemperatureLogitsWarper(0.8),
    TopKLogitsWarper(50),
    TopPLogitsWarper(0.95),
])

processors = LogitsProcessorList([
    RepetitionPenaltyLogitsProcessor(penalty=1.2)
])


def greedy_generate(model, tokenizer, input_ids, max_new_tokens=30, device="cpu"):
    model.eval()
    generated = input_ids.to(device)

    for _ in range(max_new_tokens):
        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids=generated)
            logits = outputs.logits  # (B, T, V)
        
        # Get last token logits
        next_token_logits = logits[:, -1, :]  # (B, V)

        next_token_logits = processors(generated, logits[:, -1, :])
        # next_token_logits = warpers(generated, next_token_logits)

        probs = F.softmax(next_token_logits, dim=-1)
            # Sample on CPU to avoid MPS issues
        next_token_id = torch.multinomial(probs.cpu(), num_samples=1).to(device)

        # Append to sequence
        generated = torch.cat([generated, next_token_id], dim=1)

        # Stop if EOS generated
        if tokenizer.eos_token_id is not None and (next_token_id == tokenizer.eos_token_id).all():
            break

    return generated

# Example usage
tokens = tokenizer("Who is USA President ?", return_tensors="pt")
output_ids = greedy_generate(model, tokenizer, tokens["input_ids"], max_new_tokens=30)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


Who is USA President ?

We have a daily Washington Post poll, released last week, about what the best meanchiet does Sept. 11 way, but you must


In [1]:
from collections import Counter

def majority_baseline(lists):
    counts = Counter(tuple(lst) for lst in lists)
    most_common, _ = counts.most_common(1)[0]
    return list(most_common)

def _align_and_issues(reference, current):
    a = reference
    b = current
    n, m = len(a), len(b)

    # DP for edit distance
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        dp[i][0] = i
    for j in range(1, m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost_sub = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(dp[i-1][j-1] + cost_sub, dp[i-1][j] + 1, dp[i][j-1] + 1)

    # Backtrace to produce alignment ops
    i, j = n, m
    ops = []
    while i > 0 or j > 0:
        if i > 0 and j > 0:
            cost_sub = 0 if a[i-1] == b[j-1] else 1
            if dp[i][j] == dp[i-1][j-1] + cost_sub:
                if cost_sub == 0:
                    ops.append(('equal', i-1, j-1, a[i-1], b[j-1]))
                else:
                    ops.append(('replace', i-1, j-1, a[i-1], b[j-1]))
                i -= 1; j -= 1
                continue
        if i > 0 and dp[i][j] == dp[i-1][j] + 1:
            ops.append(('delete', i-1, None, a[i-1], None))
            i -= 1
            continue
        if j > 0 and dp[i][j] == dp[i][j-1] + 1:
            ops.append(('insert', None, j-1, None, b[j-1]))
            j -= 1
            continue
        # Fallback (shouldn't happen)
        raise RuntimeError("Backtrace failed")

    ops.reverse()

    # Turn alignment ops into issues
    issues = []
    total_missing = 0
    for tag, i_idx, j_idx, aval, bval in ops:
        if tag == 'equal':
            continue
        if tag == 'replace':
            # one-to-one substitution -> treat as swap/substitution
            issues.append(('swap', i_idx, aval, bval))
        elif tag == 'delete':
            issues.append(('missing', i_idx, aval))
            total_missing += 1
        elif tag == 'insert':
            issues.append(('extra', j_idx, bval))

    return issues, total_missing

def compare_to_reference(lists, reference):
    results = []
    for idx, current in enumerate(lists):
        issues, total_missing = _align_and_issues(reference, current)
        results.append((idx, issues, total_missing))
    return results


# -------------------
# Quick demo with your failing cases
# -------------------
if __name__ == "__main__":
    examples = [
        [
            ["FlexContainer", "FlexContainer", "Button"],
            ["FlexContainer", "FlexContainer", "FlexContainer"],
            ["FlexContainer", "FlexContainer", "FlexContainer"],
        ]

       
    ]

# -------------------
# Tests
# -------------------

for ex in examples:
    baseline = majority_baseline(ex)
    print("Baseline:", baseline)
    out = compare_to_reference(ex, baseline)
    for line in out:
        print(line)
    print("-" * 40)


Baseline: ['FlexContainer', 'FlexContainer', 'FlexContainer']
(0, [('swap', 2, 'FlexContainer', 'Button')], 0)
(1, [], 0)
(2, [], 0)
----------------------------------------


In [16]:
tmp = {"a":1, "b":2, "c":3}

In [18]:
tmp.values(), tmp.keys()

(dict_values([1, 2, 3]), dict_keys(['a', 'b', 'c']))

In [23]:
tmp = [1,2,3,4]

index = 0
current = tmp[index] if index >= 0 else None 
previous = tmp[index-1] if index-1>=0 else None 
next = tmp[index+1] if index+1<len(tmp) else None 

current, previous, next

(1, None, 2)